In [590]:
import itertools
import pdb
import os
from helper import *

In [591]:
def findsubsets(s, n): 
    #return list(itertools.combinations(s, n)) 
    return list(map(set, itertools.combinations(s, n))) 
s = {1, 2, 3} 
n = 2
  
print(findsubsets(s, n))

[{1, 2}, {1, 3}, {2, 3}]


In [592]:
def mes(apelido_fundo,anomes,tipo):
    if tipo == 'acoes':
        aux = pd.read_csv('excel_fundos\\'+apelido_fundo+'\\'+str(anomes)+'.xls',index_col = 0,sep='\t',decimal= '.')
        lista_acoes = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                         'Ações e outros TVM cedidos em empréstimo',
                         'Ações',
                         'Valores a pagar']
        
        aux.pf_quant = aux.pf_quant.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
        aux.pf_valor_mercado = aux.pf_valor_mercado.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
        #aux.pf_quant = aux.pf_quant*abs(aux.porcent_partrimonio_liquido)/aux.porcent_partrimonio_liquido
        #aux.pf_valor_mercado = aux.pf_valor_mercado*abs(aux.porcent_partrimonio_liquido)/aux.porcent_partrimonio_liquido
        return aux[aux.ativo1.isin(lista_acoes)]

In [593]:
def precos(anomes,ativo):
    precos = pd.read_csv(str(anomes)[:-2]+'.txt',index_col=False,sep=';',header=0,chunksize=10000).read()
    precos = precos[(precos.CODNEG == ativo)&(precos.TPMERC == 10.0)&(precos.DATA > 100*int(anomes))&(precos.DATA < 100+100*int(anomes))]
    return precos

In [594]:
def teste(carteira_0,carteira_1,ativo,anomes):
    if anomes[4:6] == '01':
        fechamento_0 = precos(str(int(anomes)-101+12),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    else:
        fechamento_0 = precos(str(int(anomes)-1),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    quantidade_referencia =100*( carteira_0[carteira_0.ativo3 == ativo].pf_quant.sum() - carteira_1[~carteira_1.fechado][carteira_1.ativo3 == ativo].pf_quant.sum() )
    auxiliar = carteira_1[carteira_1.fechado][carteira_1.candidatos.str.contains(ativo)][['ativo1','aserestudado','vendas_montante','aquis_montante','pf_valor_mercado','cand_pf_quant']].fillna(0).copy()
    
    preco = precos(anomes,ativo)
    maximo = max(preco.PREMAX)
    minimo = min(preco.PREMIN)
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    
    #auxiliar['MAX'] = 100*auxiliar.cand_pf_quant +(auxiliar.vendas_montante+99)/minimo- auxiliar.aquis_montante/maximo
    auxiliar['MAX'] = (auxiliar.pf_valor_mercado+99)/fechamento_1 +(auxiliar.vendas_montante+99)/minimo - auxiliar.aquis_montante/maximo
    #auxiliar['MIN'] = 100*auxiliar.cand_pf_quant  +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    auxiliar['MIN'] = auxiliar.pf_valor_mercado/fechamento_1 +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    lista = []
    #print(fechamento_0,minimo,maximo,fechamento_1,quantidade_referencia,"\n",auxiliar)
    #pdb.set_trace()
    for k in range(1,len(auxiliar.index)+1):
        for j in findsubsets(auxiliar.index, k):
            if round(auxiliar[auxiliar.index.isin(j)].MAX.sum()) >= quantidade_referencia >= round(auxiliar[auxiliar.index.isin(j)].MIN.sum()):
                if len(auxiliar[auxiliar.index.isin(j)].ativo1) == len(set(auxiliar[auxiliar.index.isin(j)].ativo1)):
                    lista.append(j)
    
    if len(lista) == 1:
        print('-------------------'+ativo+'-------------------')
        
    else:
        print(ativo,quantidade_referencia,"\n",auxiliar,"\n",lista)
    carteira_1 = completa(ativo,lista[0],carteira_1)
    return carteira_1

In [595]:
def teste_fraco(carteira_0,MES,ativo,anomes):
    carteira_1 =MES.copy()
    if anomes[4:6] == '01':
        fechamento_0 = precos(str(int(anomes)-101+12),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    else:
        fechamento_0 = precos(str(int(anomes)-1),ativo).sort_values(by='DATA',ascending=False).iloc[0,10]
    quantidade_referencia =100*( carteira_0[carteira_0.ativo3 == ativo].pf_quant.sum() - carteira_1[~carteira_1.fechado][carteira_1.ativo3 == ativo].pf_quant.sum() )
    auxiliar = carteira_1[carteira_1.fechado][carteira_1.candidatos.str.contains(ativo)][['ativo1','aserestudado','vendas_montante','aquis_montante','pf_valor_mercado','cand_pf_quant']].fillna(0).copy()
    
    preco = precos(anomes,ativo)
    maximo = max(preco.PREMAX)
    minimo = min(preco.PREMIN)
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    
    #auxiliar['MAX'] = 100*auxiliar.cand_pf_quant +(auxiliar.vendas_montante+99)/minimo- auxiliar.aquis_montante/maximo
    auxiliar['MAX'] = (auxiliar.pf_valor_mercado+99)/fechamento_1 +(auxiliar.vendas_montante+99)/minimo - auxiliar.aquis_montante/maximo
    #auxiliar['MIN'] = 100*auxiliar.cand_pf_quant  +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    auxiliar['MIN'] = auxiliar.pf_valor_mercado/fechamento_1 +auxiliar.vendas_montante/maximo - (auxiliar.aquis_montante+99)/minimo 
    lista = []
    #print(fechamento_0,minimo,maximo,fechamento_1,quantidade_referencia,"\n",auxiliar)
    #pdb.set_trace()
    for k in range(1,len(auxiliar.index)+1):
        for j in findsubsets(auxiliar.index, k):
            if round(auxiliar[auxiliar.index.isin(j)].MAX.sum()) >= quantidade_referencia >= round(auxiliar[auxiliar.index.isin(j)].MIN.sum()):
                if len(auxiliar[auxiliar.index.isin(j)].ativo1) == len(set(auxiliar[auxiliar.index.isin(j)].ativo1)):
                    lista.append(j)
    
    if len(lista) == 1:
        print('-------------------'+ativo+'-------------------')
        carteira_1 = completa(ativo,lista[0],carteira_1)
    else:
        print(ativo,quantidade_referencia,"\n",auxiliar,"\n",lista)
    return carteira_1

In [596]:
def filtro_alfabetico(carteira,alfabetico,aserestudado_com_varios_ativo3=[]):
    if type(aserestudado_com_varios_ativo3) != list:
        aserestudado_com_varios_ativo3 = [aserestudado_com_varios_ativo3]
    teste = carteira.copy()
    for aserestudado,ativo3,grupo in teste[~teste.ativo3.isnull()][~teste.aserestudado.isin(aserestudado_com_varios_ativo3)][['aserestudado','ativo3','grupo']].values:
    #for aserestudado,ativo3,grupo in teste[['aserestudado','ativo3','grupo']].values:
        if alfabetico:
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado>aserestudado)&(teste.candidatos<=ativo3)].index
            teste.drop(list(dropar), inplace=True)
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado<aserestudado)&(teste.candidatos>=ativo3)].index
            teste.drop(list(dropar), inplace=True)
        else:
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado>aserestudado)&(teste.candidatos>=ativo3)].index
            teste.drop(list(dropar), inplace=True)
            dropar = teste[(teste.grupo==grupo)&(teste.aserestudado<aserestudado)&(teste.candidatos<=ativo3)].index
            teste.drop(list(dropar), inplace=True)
    return teste
    

In [597]:
def filtro_unico_index(carteira,aserestudado_sem_nenhum_ativo3=[]):
    carteira = carteira.copy()
    if type(aserestudado_sem_nenhum_ativo3) != list:
        aserestudado_sem_nenhum_ativo3 = [aserestudado_sem_nenhum_ativo3]
    CONFUSAO = carteira[~carteira.aserestudado.isin(aserestudado_sem_nenhum_ativo3)].groupby(by='aserestudado').count()[['ativo3']]
    if (CONFUSAO.ativo3==1).all():
        print('trabalho concluido'.upper())
        return carteira[(carteira.ativo3.notnull())|(carteira.aserestudado.isin(aserestudado_sem_nenhum_ativo3))].sort_values(['aserestudado','candidatos'])
    else:
        
        print(CONFUSAO[CONFUSAO.ativo3>1])
        print(CONFUSAO[CONFUSAO.ativo3==0])
        return carteira.sort_values(['aserestudado','candidatos'])

In [598]:
def separa_blocos(carteira_pandas):
    
    
    
    teste = carteira_pandas.sort_values(by='aserestudado').copy()


    assert len(teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].index )==1
    assert len(teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].index )==1



    alfabetico = teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].aserestudado.values[0] < teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].aserestudado.values[0] 

    if not alfabetico:
        teste =teste.sort_values(by=['candidatos'],ascending=False).sort_values(by=['aserestudado'])
    
    
    assert len(teste[teste.ativo1.str.upper() == 'Obrigações por ações e outros TVM recebidos em empréstimo'.upper()].index )==0
    assert len(teste[teste.ativo1.str.upper() == 'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública'.upper()].index )==0
    assert len(teste[teste.ativo1.str.upper() == 'Valores a pagar'.upper()].index )<=1

    disponibilidade = teste[teste.ativo1.str.upper() == 'disponibilidades'.upper()].aserestudado.values[0]
    valor_receber =teste[teste.ativo1.str.upper() == 'Valores a receber'.upper()].aserestudado.values[0]


    teste['grupo'] = 0
    
    if not alfabetico:
        print('ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)')
        print('Tento by passar usando a localização de algum Investimento no Exterior')
        #return

        investimento_exterior_aserestudado =teste[teste.ativo1.str.upper() == 'Investimento no Exterior'.upper()].aserestudado.values[0]



        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar']
        grupo_1 = teste[teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 1

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 2





        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar',
                            'Brazilian Depository Receipt - BDR']
        grupo_1 = teste[teste.aserestudado>investimento_exterior_aserestudado][teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 3

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado>investimento_exterior_aserestudado][teste.aserestudado<valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 4

    else:
        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar']
        grupo_1 = teste[teste.aserestudado<disponibilidade][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 1

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado<disponibilidade][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 2





        #Aqui considero que se houver mais de um valor a pagar, ele vaão estar separados
        lista = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                            'Outros valores mobiliÃ¡rios registrados na CVM objeto de oferta pública',
                            'Ações',
                            'Certificado ou recibo de depósito de valores mobiliários',
                            'Valores a pagar',
                            'Brazilian Depository Receipt - BDR']
        grupo_1 = teste[teste.aserestudado>valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 3

        lista = ['Ações e outros TVM cedidos em empréstimo']
        grupo_1 = teste[teste.aserestudado>valor_receber][teste.ativo1.isin(lista)].index
        teste.loc[grupo_1,'grupo']= 4





    for t in set(teste.grupo) :
        if t != 0 and teste[teste.grupo==t].ativo3.apply(pd.isna).all() :
            print('grupo',t,'todos os ativos1 são nulos')

    return teste,alfabetico

In [599]:
anomes = 202004
apelido_fundo = 'const'.upper()


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.23 s


In [600]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)
carteira_pandas[carteira_pandas.grupo == 1]
aux = carteira_pandas.copy()

True

In [601]:
anomes = 202005


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.24 s


In [602]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

ISSO ESTA ERRADO, QUANDO É NÃO ALFABETICO (olhar squadra 202004 o aserestudado 58331405 q vc vai entender)
Tento by passar usando a localização de algum Investimento no Exterior
grupo 3 todos os ativos1 são nulos


In [603]:
set(carteira_pandas[carteira_pandas.grupo == 0].ativo1)

{'Cotas de Fundos',
 'Disponibilidades',
 'Investimento no Exterior',
 'Valores a pagar',
 'Valores a receber'}

In [604]:
def explodir(carteira_pandas,alfabetico):
    teste = carteira_pandas[carteira_pandas.grupo!=0]
    assert len(teste[teste.candidatos.isnull()]) ==0
    teste = pd.DataFrame(columns=['index','aserestudado','ativo1','ativo2','ativo3','ativo4','classificacao','emp_ligada','vendas_quant','vendas_montante','aquis_quant','aquis_montante','pf_quant','pf_valor_custo','pf_valor_mercado','porcent_partrimonio_liquido','fechado','candidatos','cand_pf_quant', 'grupo'],data=[(index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,c,q,grupo) if pd.isnull(ativo3) else (index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,ativo3,pf_quant,grupo) for (index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo) in teste.reset_index().values  for (c,q) in zip(candidatos,cand_pf_quant) ]).set_index('index')
    teste = teste.drop_duplicates().sort_values(by='aserestudado').reset_index().rename(columns={'index':'position'})
    

    teste = filtro_alfabetico(teste,alfabetico)
    teste.pf_quant = teste.cand_pf_quant
    
    
    teste.pf_quant = teste.pf_quant.apply(abs)*teste.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
    teste.pf_valor_mercado = teste.pf_valor_mercado.apply(abs)*teste.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
    
    
    
    return teste

In [605]:
explodido = explodir(carteira_pandas,alfabetico)

In [606]:
def completa(ativo,linhas,teste):
    for i in linhas:
        teste.loc[int(i),'ativo3'] = ativo
    return teste

In [607]:
lista_acoes = ['Obrigações por ações e outros TVM recebidos em empréstimo',
                 'Ações e outros TVM cedidos em empréstimo',
                 'Ações',
                 'Certificado ou recibo de depósito de valores mobiliários', 
                 'Valores a pagar']

aux.pf_quant = aux.pf_quant.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
aux.pf_valor_mercado = aux.pf_valor_mercado.apply(abs)*aux.ativo1.apply(lambda x: -1 if 'VALORES A PAGAR' in x.upper() or 'OBRIGAÇÕES' in x.upper() else 1)
aux =  aux[aux.ativo1.isin(lista_acoes)]


In [608]:
aux.ativo3.unique()

array(['ARZZ3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'GNDI3', 'LREN3',
       'MGLU3', 'PSSA3', 'RADL3', 'RAIL3', 'RENT3', 'SULA11', 'TOTS3',
       'VIVA3', 'WEGE3', 'B3SA3', 'LAME3', 'LAME4', 'NTCO3', 'SUZB3',
       'TEND3', 'Valores a Pagar', 'PAT/BPAC11/BRBPACUNT006'],
      dtype=object)

In [609]:
aux.ativo3 = aux.ativo3.apply(lambda x : x if x != 'PAT/BPAC11/BRBPACUNT006' else 'BPAC11')

''

In [610]:
MES0 = aux.copy()
MES1 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES1 = teste_fraco(MES0,MES1,ativo,'202005')

-------------------ARZZ3-------------------
B3SA3 -2320000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BPAC11-------------------
-------------------ENEV3-------------------
ENGI11 335150300.0 
                                                  ativo1  aserestudado  \
1314  Certificado ou recibo de depósito de valores m...      58845430   
2367           Ações e outros TVM cedidos em empréstimo      58845447   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
1314              0.0             0.0       12385264000      2607424.0   
2367              0.0             0.0        4016975200       845679.0   

               MAX           MIN  
1314  2.607424e+08  2.607424e+08  
2367  8.456790e+07  8.456790e+07   
 []
EQTL3 406410000.0 
                                         ativo1  aserestudado  vendas_montante  \
743                               

In [611]:
MES1 = completa('ENGI11',{1314,2367},MES1)
MES1 = completa('KLBN11',{1007},MES1)
MES1 = completa('TOTS3',{183,1720},MES1) #Teve aumento de capital
MES1 = completa('RENT3',{377,1755},MES1) 
MES1 = completa('EQTL3',{2297,1251},MES1) 
MES1 = completa('EZTC3',{2215,1204},MES1) 
MES1 = completa('PSSA3',{506,1883},MES1) 
MES1 = completa('ALPK3',{1507},MES1) 
MES1 = completa('ALUP11',{2659},MES1) 

In [612]:
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [613]:
#papeis que ele Não achou
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [614]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES1 = filtro_alfabetico(MES1,alfabetico)
fechados_vou_abrir = MES1[~MES1.ativo3.isnull()].index
MES1.loc[fechados_vou_abrir,'fechado'] = False

In [615]:

assert len(CONFUSAO[CONFUSAO.ativo3>1]) != 0
MES1 = filtro_alfabetico(MES1,alfabetico,aserestudado_com_varios_ativo3=[58881799,58881816,58881820])

AssertionError: 

In [482]:
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES1 = teste_fraco(MES0,MES1,ativo,'202005')

ALPA4 3349900.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
ALSO3 -644200.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
B3SA3 -1581500.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
BBAS3 300000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
BBDC4 3150000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
BPAC11 7314300.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
CSAN3 -1998600.0 
 Empty DataFrame
Columns: [ativo1, aser

In [616]:
#papeis que ele Não achou
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [491]:
 MES1[MES1.aserestudado == 58877078][MES1.candidatos == 'SULA11']

,position,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1509,26,58877078,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,139350,NaN,624984700,2.0,True,SULA11,139350,1


In [492]:
MES1 = completa('SULA11',{1509},MES1)

In [617]:
#Não achou
nao_achou = MES1[MES1.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]
nao_achou

,position,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo


In [496]:
MES0[MES0.ativo1.str.contains('Certificado')]

,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
27,58387424,Certificado ou recibo de depósito de valores m...,Cod. Ativo:,SULA11,Dt. Ini. Vigen.:05/10/2007,Para negociacao,Nao,NaN,NaN,NaN,NaN,91523.0,NaN,409565425,2.048,False,"[A1LL34, ALUP4, ATOM3, AZEV4, BBRK3, BEES3, BI...","[69654, 642960, 3056459, 1489329, 1706523, 809...",1


In [431]:
MES1[MES1.candidatos=='SULA11']

,position,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
1509,26,58877078,Certificado ou recibo de depósito de valores m...,NaN,NaN,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,139350,NaN,624984700,2.0,True,SULA11,139350,1


In [432]:
MES1 = completa('SULA11',{1509},MES1)

In [497]:
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [618]:
list(CONFUSAO[CONFUSAO.ativo3<1].index)

[]

In [499]:
MES1 = filtro_alfabetico(MES1,alfabetico)
CONFUSAO = MES1.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [619]:
MES1 = filtro_unico_index(MES1,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES1 = MES1.set_index('index')

TRABALHO CONCLUIDO


In [68]:
#MES1[MES1.aserestudado==58334057]

In [69]:
#MES1[MES1.aserestudado==58334058]

''

In [438]:
''

''

In [620]:
#MES1.sort_values(['aserestudado','candidatos']).to_csv('excel_fundos\\'+str(anomes)+'_2.xls',sep='\t',decimal= '.')

In [621]:
#aqui tenho q ver se as linhas com short estao realmente negativas
aux = MES1.copy()

In [622]:
anomes = 202006

%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.31 s


In [623]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

grupo 3 todos os ativos1 são nulos
grupo 4 todos os ativos1 são nulos


In [624]:
explodido = explodir(carteira_pandas,alfabetico)

In [625]:
MES1 = aux.copy()
MES2 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES2 = teste_fraco(MES1,MES2,ativo,'202006')

-------------------ALPK3-------------------
ALUP11 3478200.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
ARZZ3 92080000.0 
                                        ativo1  aserestudado  vendas_montante  \
134  Ações e outros TVM cedidos em empréstimo      59602657              0.0   

     aquis_montante  pf_valor_mercado  cand_pf_quant           MAX  \
134             0.0        2515487600       525154.0  5.251540e+07   

              MIN  
134  5.251540e+07   
 []
B3SA3 117750000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BPAC11-------------------
-------------------ENEV3-------------------
ENGI11 345310300.0 
                                                  ativo1  aserestudado  \
85             Ações e outros TVM cedidos em empréstimo      59602656   
1246  Certificado

In [631]:
MES2 = completa('ENGI11',{85,1246},MES2)
MES2 = completa('KLBN11',{1412},MES2)
MES2 = completa('LAME4',{336,1590},MES2)


In [632]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [633]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [634]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES2 = filtro_alfabetico(MES2,alfabetico)

In [635]:
#Não achou
nao_achou = MES2[MES2.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]
nao_achou

,position,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo


In [92]:
MES1[MES1.ativo1.str.contains('Certificado')]

,index,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo
89,33,58881799,Certificado ou recibo de depósito de valores m...,NaN,SULA11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1359000.0,NaN,6095115000,2.0,True,SULA11,1359000.0,1
1097,34,58881816,Certificado ou recibo de depósito de valores m...,NaN,ENGI11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1074527.0,NaN,5104003200,1.0,True,ENGI11,1074527.0,1
1375,35,58881820,Certificado ou recibo de depósito de valores m...,NaN,BIDI11,NaN,Para negociacao,Nao,NaN,NaN,NaN,NaN,1404488.0,NaN,4719079600,1.0,True,BIDI11,1404488.0,1


In [514]:
MES2 = completa('MRVE3',{1065 },MES2)

In [515]:
#MES2 = teste_fraco(MES1,MES2,'BIDI4',str(anomes))

In [516]:
CONFUSAO = MES2.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [636]:
MES2 = filtro_unico_index(MES2,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES2.set_index('index')

TRABALHO CONCLUIDO


In [637]:
#aqui tenho q ver se as linhas com short estao realmente negativas
aux = MES2.copy()

In [638]:
anomes = 202007


%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

Wall time: 1.26 s


In [639]:
carteira_pandas,alfabetico = separa_blocos(carteira_pandas)

grupo 3 todos os ativos1 são nulos
grupo 4 todos os ativos1 são nulos


In [640]:
set(carteira_pandas[carteira_pandas.grupo == 0].ativo1)

{'Cotas de Fundos',
 'Disponibilidades',
 'Investimento no Exterior',
 'Valores a pagar',
 'Valores a receber'}

In [641]:
explodido = explodir(carteira_pandas,alfabetico)

202007

In [642]:
MES2 = aux.copy()
MES3 = explodido.copy()
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES3 = teste_fraco(MES2,MES3,ativo,'202007')

-------------------ALPK3-------------------
B3SA3 -110370000.0 
                                        ativo1  aserestudado  vendas_montante  \
325  Ações e outros TVM cedidos em empréstimo      59976717              0.0   

     aquis_montante  pf_valor_mercado  cand_pf_quant           MAX  \
325             0.0        2030733400       320103.0  3.201030e+07   

              MIN  
325  3.201030e+07   
 []
-------------------BPAC11-------------------
-------------------ENEV3-------------------
ENGI11 354820300.0 
                                                  ativo1  aserestudado  \
121            Ações e outros TVM cedidos em empréstimo      59976713   
1117  Certificado ou recibo de depósito de valores m...      59976733   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
121               0.0             0.0        3659512400       744560.0   
1117              0.0             0.0       14726534300      2996243.0   

               MAX           MIN  


In [675]:
MES3 = completa('ENGI11',{1117,121  },MES3)
MES3 = completa('KLBN11',{1283  },MES3)
MES3 = completa('SOMA3',{1829  },MES3)
MES3 = completa('BPAC11',{2198  },MES3)


In [557]:
MES3 = teste_fraco(MES2,MES3,'GGBR3',str(anomes))

-------------------GGBR3-------------------


In [677]:
CONFUSAO = MES3.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3>1]

,ativo3
aserestudado,


In [678]:
#papeis que ele Não achou
CONFUSAO = MES3.groupby(by='aserestudado').count()[['ativo3']]
CONFUSAO[CONFUSAO.ativo3<1]

,ativo3
aserestudado,


In [679]:
nao_achou = MES3[MES3.aserestudado.isin(CONFUSAO[CONFUSAO.ativo3<1].index)]
nao_achou

,position,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant,grupo


In [680]:
#Antes de rodar o filtro_alfabetico preciso me certicicar que não há nenhuma linha com mais de um valor na coluna ativo3, ou seja nenhum aserestudado_com_varios_ativo3, ou seja, que  len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
assert len(CONFUSAO[CONFUSAO.ativo3>1]) == 0
MES3 = filtro_alfabetico(MES3,alfabetico)
fechados_vou_abrir = MES1[~MES1.ativo3.isnull()].index
MES1.loc[fechados_vou_abrir,'fechado'] = False

In [681]:
MES3 = filtro_unico_index(MES3,aserestudado_sem_nenhum_ativo3=list(CONFUSAO[CONFUSAO.ativo3<1].index))
#MES2.set_index('index')

TRABALHO CONCLUIDO


In [649]:
for ativo in sorted(list({x for x in aux.ativo3 if pd.notna(x)})):
    if ativo.upper() not in ('MRSA5B','MRSA3B','MOAR3','EUCA4','USIM6','TIET12','Valores a Pagar'.upper()):
        MES3 = teste_fraco(MES2,MES3,ativo,'202007')

-------------------ALPK3-------------------
B3SA3 -110370000.0 
 Empty DataFrame
Columns: [ativo1, aserestudado, vendas_montante, aquis_montante, pf_valor_mercado, cand_pf_quant, MAX, MIN]
Index: [] 
 []
-------------------BPAC11-------------------
-------------------ENEV3-------------------
ENGI11 354820300.0 
                                                  ativo1  aserestudado  \
121            Ações e outros TVM cedidos em empréstimo      59976713   
1117  Certificado ou recibo de depósito de valores m...      59976733   

      vendas_montante  aquis_montante  pf_valor_mercado  cand_pf_quant  \
121               0.0             0.0        3659512400       744560.0   
1117              0.0             0.0       14726534300      2996243.0   

               MAX           MIN  
121   7.445600e+07  7.445600e+07  
1117  2.996243e+08  2.996243e+08   
 []
-------------------EQTL3-------------------
-------------------GNDI3-------------------
KLBN11 419520000.0 
                         

In [650]:
#MES3[MES3.aserestudado==60010409]

In [666]:
MES3 =teste_fraco(MES2,MES3,'ALPA4','202007')

-------------------ALPA4-------------------


In [542]:
for k in nao_achou.aserestudado.unique():
    for cand in MES3[MES3.aserestudado==k].candidatos.values:
        teste_fraco(MES2,MES3,cand,'202007')

-------------------BBAS3-------------------
-------------------ETER9-------------------
EUCA4 0 
     ativo1  aserestudado  vendas_montante  aquis_montante  pf_valor_mercado  \
643  Ações      60010409       12782800.0     258724300.0         273975000   

     cand_pf_quant           MAX           MIN  
643         434881  8.868149e+06  3.232250e+06   
 []
FLMA11 0 
     ativo1  aserestudado  vendas_montante  aquis_montante  pf_valor_mercado  \
648  Ações      60010409       12782800.0     258724300.0         273975000   

     cand_pf_quant           MAX           MIN  
648         910216  1.845885e+07  4.991139e+06   
 []
GDBR34 0 
     ativo1  aserestudado  vendas_montante  aquis_montante  pf_valor_mercado  \
642  Ações      60010409       12782800.0     258724300.0         273975000   

     cand_pf_quant           MAX           MIN  
642           3653  57374.707627  36389.903052   
 []
-------------------GGBR3-------------------
-------------------GOAU4-------------------
GOLL11

In [189]:
MES3 = teste_fraco(MES2,MES3,'RADL3','202007')

-------------------RADL3-------------------


In [682]:
#MES0.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202004)+'.xls',sep='\t',decimal= '.')

In [683]:
#MES1.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202005)+'.xls',sep='\t',decimal= '.')

In [684]:
#MES2.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202006)+'.xls',sep='\t',decimal= '.')

In [685]:
#MES3.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(202007)+'.xls',sep='\t',decimal= '.')

''

In [698]:
anomes = 202005

%time carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=False)

Wall time: 1.23 s


In [695]:
assert len(MES1.aserestudado) == len(set(MES1.aserestudado))
for i,row in MES1.iterrows():
    assert carteira_pandas.loc[row['position'],'aserestudado'] == row['aserestudado']
    carteira_pandas.loc[row['position'],'ativo3'] = row['ativo3']
    carteira_pandas.loc[row['position'],'pf_quant'] = row['pf_quant']

    preco = precos(str(anomes),row['ativo3'])
    fechamento_1 = preco.sort_values(by='DATA',ascending=False).iloc[0,10]
    sinal = carteira_pandas.loc[row['position'],'pf_valor_mercado']/abs(carteira_pandas.loc[row['position'],'pf_valor_mercado'])
    carteira_pandas.loc[row['position'],'pf_valor_mercado'] = sinal*int(round(100*row['pf_quant']*fechamento_1))
carteira_pandas.porcent_partrimonio_liquido = carteira_pandas.pf_valor_mercado/float(pl)

In [696]:
carteira_pandas.to_csv('excel_fundos\\'+apelido_fundo+'\\'+str(anomes)+'.xls',sep='\t',decimal= '.')

In [689]:
pl

'5340155776.20'